In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
df_movies=pd.read_csv('Movies.csv')

In [3]:
df_movies['genre_name']=df_movies['genre_name'].astype(str)

In [17]:
#El modelo utilizado para el sistema de recomendación es K-Nearest Neighbors-KNN.
#La principal razón por la que se utilizó este modelo, es que el Dataframe a utilizar
#presenta las columna (usadas) 'title' y 'vote_average' y cumplieron la característica de que
#se pueden comparar directamente utilizando medidas de similitud.
#También el modelo presenta una sencillez para su implementación y encaja con el sistema
#Por útlimo, el modelo KNN es factible para manejar texto, este caso la columna 'title' y 
#es capaz de hacer recomendaciones, que al final es la esencia de lo que se quiere lograr
#Cabe mencionar que se uso una normalización en la columna 'vote_average' para reducir la influencia
#de valores extremos y con ello mejorar la precisión del modelo, en este ejercicio, se uso 0-1


#Filtramos las columnas a utilizar

df_to_use=df_movies[['title', 'vote_average']]

#Se limitaa 1500 filas esto debido a recursos de la máquina usada para crear el modelo
#Si se tienen más recursos se puede aumentar
df_used=df_to_use.sample(n=1500, random_state=42)


#Aquí se usa la normalización ya mencionada de la columna 'vote_average' en una escala de 0-1
scaler=MinMaxScaler()
df_used['vote_average_normalized'] = scaler.fit_transform(df_used[['vote_average']])

#Preprocesamos los títulos de las películas utilizando TF-IDF, con el fin de convertir texto
#en una representación numerica y asi encontrar peliculas similares
tfidf_vectorizer=TfidfVectorizer(stop_words='english')
tfidf_matrix=tfidf_vectorizer.fit_transform(df_used['title'])

#Se define el modelo K-Nearest Neighbors
knn_model=NearestNeighbors(n_neighbors=6, algorithm='auto').fit(tfidf_matrix)


def recomendacion(movie, model, tfidf_matrix, df_sample, num=5):
    
    #Se transorma el titulo ingresado en vector TF-IDF
    vectorized_movie= tfidf_vectorizer.transform([movie])
    
    #Se buscan las peliculas cercanas que cumplen con las características propuestas
    d, i=model.kneighbors(vectorized_movie, n_neighbors=num+1)
    
    #Se obtienen las peliculas con sus titulos y vote_average similares al titulo de entrada, excluyendo al mismo
    recom=df_used.iloc[i[0][1:num+1]].copy()
    return recom[['title', 'vote_average', 'vote_average_normalized']]


movie="Toy Story"
recomendacion(movie, knn_model, tfidf_matrix, df_used, num=5)

,title,vote_average,vote_average_normalized
2718,The Story of G.I. Joe,4.8,0.452632
12117,The Tillman Story,7.7,0.757895
2842,The Official Story,7.3,0.715789
499,Two Much,5.1,0.484211
16894,C.O.G.,5.4,0.515789
